In [ ]:
import pandas as pd
import sqlite3

# Read final dataframe

In [ ]:
df = pd.read_pickle('./df_pm_oa_preprocessed.pkl')

# Make SQL DB

In [ ]:
# Convert list-type columns to strings
for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, list)).any():
        df[col] = df[col].apply(lambda x: str(x) if isinstance(x, list) else x)

In [ ]:
conn = sqlite3.connect("data.db")

df.to_sql("publications", conn, if_exists="replace", index=False)

conn.close()

# Optimize data types in DB and save

In [ ]:
DB_FILE = "data.db"

conn = sqlite3.connect(DB_FILE)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS publications_optimized (
    pmid INTEGER PRIMARY KEY,
    year_pubmed INTEGER,
    journal_title TEXT,
    title_pubmed TEXT,
    cited_by_count SMALLINT,
    countries TEXT,
    authors_number SMALLINT,
    is_open_access BOOLEAN,
    majority_country TEXT,
    cit_per_year FLOAT(4),
    n_references SMALLINT
);
""")

cursor.execute("""
INSERT INTO publications_optimized
SELECT * FROM publications;
""")

cursor.execute("CREATE INDEX IF NOT EXISTS idx_year_pubmed ON publications_optimized(year_pubmed);")
cursor.execute("CREATE INDEX IF NOT EXISTS idx_pmid ON publications_optimized(pmid);")

cursor.execute("DROP TABLE IF EXISTS publications;")
cursor.execute("ALTER TABLE publications_optimized RENAME TO publications;")

conn.commit()  

cursor.execute("VACUUM;")

conn.close()